In [1]:
DATA_NAME = 'coco-indoor' 
TRANSFORM = 'wavelet-horizontal'
CHANNEL = 'blue'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_blue_horizontal_wavelet_indoor_coco.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,47689.14,0.87,0.23,0.15,0.01,0.00,-0.00,-0.00
3,0.87,9767.03,-0.09,0.06,-0.00,0.00,-0.00,0.00
4,0.23,-0.09,1821.08,0.01,0.00,-0.00,0.00,-0.00
5,0.15,0.06,0.01,284.76,0.00,0.00,0.00,-0.00
6,0.01,-0.00,0.00,0.00,40.82,0.00,0.00,0.00
7,0.00,0.00,-0.00,0.00,0.00,5.44,-0.00,0.00
8,-0.00,-0.00,0.00,0.00,0.00,-0.00,0.69,0.00
9,-0.00,0.00,-0.00,-0.00,0.00,0.00,0.00,0.07


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,0.00004,0.00002,0.00004,0.00000,0.00000,-0.00000,-0.00001
3,0.00004,1.00000,-0.00002,0.00003,-0.00000,0.00001,-0.00003,0.00003
4,0.00002,-0.00002,1.00000,0.00002,0.00001,-0.00004,0.00004,-0.00002
5,0.00004,0.00003,0.00002,1.00000,0.00001,0.00002,0.00001,-0.00002
6,0.00000,-0.00000,0.00001,0.00001,1.00000,0.00002,0.00002,0.00002
7,0.00000,0.00001,-0.00004,0.00002,0.00002,1.00000,-0.00003,0.00001
8,-0.00000,-0.00003,0.00004,0.00001,0.00002,-0.00003,1.00000,0.00002
9,-0.00001,0.00003,-0.00002,-0.00002,0.00002,0.00001,0.00002,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

1.292763305236042

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[2.84541608e+08 1.16827503e+07 3.94798460e+05 9.26985871e+03
 1.78677119e+02 2.82462444e+00 3.38598553e-02 1.04787066e-34]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.999521,-0.030466,-0.005416,-0.000843,-0.000121,-0.000016,-0.000002,-2.319096e-07
1,0.030271,0.999007,-0.032335,-0.004848,-0.000696,-0.000092,-0.000012,-1.176185e-06
2,0.006360,0.031981,0.998940,-0.032178,-0.004480,-0.000599,-0.000075,-8.361174e-06
3,0.001188,0.005815,0.031812,0.998792,-0.036677,-0.004763,-0.000600,-6.580141e-05
4,0.000213,0.001041,0.005583,0.036255,0.998284,-0.045284,-0.005573,-6.074050e-04
5,0.000038,0.000184,0.000991,0.006321,0.044625,0.996909,-0.063982,-6.813774e-03
6,0.000007,0.000035,0.000185,0.001184,0.008265,0.062545,0.992027,-1.090924e-01
7,0.000002,0.000008,0.000041,0.000261,0.001821,0.013670,0.108433,9.940081e-01


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.00047923751298062367,
 0.0009934442704978563,
 0.0010601041618834017,
 0.0012084716623124159,
 0.0017158494950736403,
 0.003091008309222487,
 0.007973198523471448,
 0.005991924469517573]